In [2]:
import pandas as pd
from datetime import datetime, timedelta
from helpers import scale_data, series_to_supervised, split_dataset, build_predictor, fit_predictor

def merge(group, weeks):
    # this outputs a row of the dataframe
    row = {'Client' : group.Client}
    sums = group.groupby('week')
    for w in range(12):
        try:
            row['w{}'.format(w)] = sums.get_group(w).Preu_Total.sum()
        except KeyError:
            row['w{}'.format(w)] = 0
            
    # row['Tipus_Client'] = group.Tipus_Client
    
    try:
        row['target'] = sums.get_group(14).Preu_Total.sum()
    except KeyError:
        row['target'] = 0
    df = pd.DataFrame(row).iloc[0]
    return df

def preprocess(DATASET_PATH):
    df = pd.read_csv(DATASET_PATH,
                     parse_dates=['Dia_Comanda'],
                     date_parser=lambda d: datetime.strptime(d, '%Y-%m-%d'))
    df = df.drop(['Provincia', 'Linia', 'Pes_mig_G', 'Mesura'], axis=1)
    min_date = df['Dia_Comanda'].min()
    df['week'] = df['Dia_Comanda'].map(lambda e: (e.year - min_date.year) * 54 + e.isocalendar()[1])
    df = df[df['week'] > df['week'].max() - PAST_TIMESTEPS-3]
    df['week'] = df['week'].map(lambda x: x - df['week'].min())
    df = df.groupby('Client').apply(lambda x: merge(x, PAST_TIMESTEPS))
    return df

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
TRAINSET_FILE = 'dades_train_P1.csv'
TESTSET_FILE = 'dades_test.csv'

PAST_TIMESTEPS = 12
FUTURE_TIMESTEPS = 3
FEATURES = 1
EPOCHS = 10
BATCH_SIZE = 100


df_train = preprocess(TRAINSET_FILE)
df_test = preprocess(TESTSET_FILE)

df_train = df_train.drop(['Client'], axis=1)
df_test = df_test.drop(['Client'], axis=1)

In [76]:
df_train.head()

,w0,w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,target
Client,,,,,,,,,,,,,
70003,19.883756,14.476900,46.314754,0.000000,23.169598,33.010432,0.000000,17.350820,12.373174,33.458718,0.000000,10.808346,0.000000
70005,3.605365,0.990164,5.927869,3.459314,7.666170,3.279285,5.360358,3.748435,8.970492,0.000000,3.093294,5.437854,0.000000
70006,24.328465,25.707899,12.586587,19.842623,22.737407,17.328167,16.956781,13.892101,22.165127,18.605663,27.532638,21.377645,20.259314
70007,14.312370,13.769300,16.609836,11.074814,13.281073,14.455440,5.199404,22.252757,3.548137,17.616692,12.603875,16.305812,11.047988
70043,4.126975,4.252161,3.949925,0.000000,5.335917,4.977645,3.526080,3.606557,0.000000,0.000000,0.000000,14.431595,2.231297


In [9]:
from sklearn import linear_model

trainset, sc = scale_data(df_train)
testset, _ = scale_data(df_test, sc)

x_train = trainset[:, :-1]
y_train = trainset[:, -1]

x_test = testset[:, :-1]
y_test = testset[:, -1]

predictor = linear_model.LinearRegression()
predictor.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [10]:
predictions = predictor.predict(x_test)

from sklearn.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(y_test, predictions))

0.0479733852933366

In [12]:
import numpy as np
import matplotlib.pyplot as plt
a = np.zeros([389, 13])
b = np.zeros([389, 13])
a[:, 12] = predictions
b[:, 12] = y_test

predictions_original = sc.inverse_transform(a)
predictions_original.shape
y_test_original = sc.inverse_transform(b)
print(sqrt(mean_squared_error(y_test_original[:, 12], predictions_original[:, 12])))
print(y_test_original[:, 12].mean())

predictions_original[:, 12]

3.5219755747586303
0.35171232464726865


array([ 7.22608641e-01,  2.63183169e+00,  1.96794513e+00,  9.44870568e+00,
        2.58949608e+00,  9.16620896e-01,  3.84822421e-01,  8.18831998e-02,
        2.76897909e-02,  3.31037556e+00,  1.12607954e+01,  1.75364988e+00,
        5.94398236e-01,  1.10398427e+00,  6.79979043e-03,  3.61161284e-01,
        8.43480616e+00,  2.85350157e-01, -3.38339735e-01,  7.19768212e-01,
        2.27393571e+00,  3.40829697e+00, -6.70092735e-01,  9.72335199e-02,
        4.70029969e+00,  3.07445811e+00,  3.30223997e+00,  1.10937492e+01,
        1.01838577e+00,  2.35105325e+01,  1.29272925e+00,  3.44680040e+00,
        1.98167314e-01,  3.24411506e+00,  4.02948965e+00,  5.52783861e-01,
        1.16964458e+00,  1.37869051e+01, -1.08634201e-01, -2.86871512e-01,
        2.16828580e+00,  7.91815957e-01,  5.74429766e+00,  6.24829151e+00,
        4.22657261e-01, -8.60020288e-03,  8.22080752e-01, -5.68516917e-02,
       -2.06053670e-01,  1.22167177e+01,  6.68243198e+00,  9.48793168e-01,
        4.12080921e+00, -

Write results into .csv file formatted as <Client, volum>

In [ ]:
predictions_original